## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.
1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).
2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  
3. To generate vectors and split the instances into training and testing datasets at random.
4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.
5. To evaluate the performance of the classifier.

You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [1]:
import a2
from sklearn.svm import LinearSVC

In [2]:
gmbfile = open('GMB_dataset.txt', "r")
# (If running on MLT server)
# gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [3]:
inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[20:40]

,sentenceNr,word,POS,NEtag
20,1,from,IN,O
21,1,that,DT,O
22,1,country,NN,O
23,2,family,NNS,O
24,2,of,IN,O
25,2,soldier,NNS,O
26,2,kill,VBN,O
27,2,in,IN,O
28,2,the,DT,O
29,2,conflict,NN,O


## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [4]:
instances = a2.create_instances(inputdata)

In [5]:
instances[20:40]

[Class: gpe Features: ['<s1>', '<s2>', '<s3>', '<s4>', '<s5>', 'this', 'week', 'restart', 'part', 'of'],
 Class: geo Features: ['the', 'conversion', 'process', 'at', 'its', 'nuclear', 'plant', '</s1>', '</s2>', '</s3>'],
 Class: gpe Features: ['<s1>', '<s2>', '<s3>', '<s4>', '<s5>', 'official', 'say', 'they', 'expect', 'to'],
 Class: tim Features: ['sensitive', 'part', 'of', 'the', 'plant', 'after', 'an', 'iaea', 'surveillance', 'system'],
 Class: org Features: ['the', 'plant', 'wednesday', 'after', 'an', 'surveillance', 'system', 'begin', 'function', '</s1>'],
 Class: org Features: ['<s2>', '<s3>', '<s4>', '<s5>', 'The', 'surveillance', 'system', 'begin', 'function', '</s1>'],
 Class: gpe Features: ['<s5>', 'The', 'european', 'union', 'with', 'backing', 'have', 'threaten', 'to', 'refer'],
 Class: gpe Features: ['backing', 'have', 'threaten', 'to', 'refer', 'to', 'the', 'u.n.', 'security', 'council'],
 Class: org Features: ['to', 'refer', 'iran', 'to', 'the', 'to', 'the', 'u.n.', 'secu

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [6]:
bigdf = a2.create_table(instances)
bigdf[20:40]

,NE_class,vector
20,gpe,"[0.29184898987871394, -0.22289236295392845, -0..."
21,geo,"[0.1574744333056034, 0.14620880871082068, 0.03..."
22,gpe,"[0.4009647553791204, -0.35883375703800696, -0...."
23,tim,"[0.04316814481269096, 0.012291577274903483, 0...."
24,org,"[0.06821698262936284, 0.04101560676950414, 0.0..."
25,org,"[0.24514571016546235, -0.1294805791229831, -0...."
26,gpe,"[0.09509039751821673, -0.061969373219631536, 0..."
27,gpe,"[0.06741957692589423, -0.014630341345356464, 0..."
28,org,"[0.09001055701714501, 0.00776316702460329, 0.2..."
29,gpe,"[0.042546286855863466, -0.02009004152557714, 0..."


In [7]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[ 1.15686623e-01,  9.13556327e-02,  6.21339887e-02, ...,
         -1.69677231e-03,  4.10390243e-03,  2.30264464e-03],
        [ 6.68462054e-02,  2.46829473e-02,  1.74062872e-01, ...,
          1.71420470e-03,  4.85696676e-03, -1.34390771e-03],
        [ 7.17889936e-02, -5.28522162e-04,  1.74671800e-01, ...,
         -2.62026663e-03,  3.10533137e-03,  9.83691479e-03],
        ...,
        [ 8.06187604e-02,  1.95192696e-02,  2.10544327e-01, ...,
          5.64462698e-03,  2.42535396e-04, -3.71737121e-03],
        [ 9.09462190e-02, -1.11156527e-02,  2.21790052e-01, ...,
          1.24428169e-04,  2.61769994e-04,  4.23292144e-03],
        [ 6.38486540e-02, -1.91450092e-02,  1.28403265e-01, ...,
         -1.45546346e-03,  2.93120717e-03, -1.84162575e-03]]),
 566     per
 183     geo
 2130    eve
 5776    org
 1706    gpe
        ... 
 6728    per
 1794    per
 3409    geo
 5862    gpe
 4026    geo
 Name: NE_class, Length: 5538, dtype: object,
 array([[ 2.60832760e-02,  5.72399289e-0

In [8]:
len(test_y) / (len(test_y) + len(train_y))

0.1999422132331696

In [9]:
len(test_X) / (len(test_X) + len(train_X))

0.1999422132331696

In [10]:
# test_y[0] # *key error if 0th row not in test dada
zeroth = test_y[0] if 0 in test_y else train_y[0]
zeroth

'geo'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [11]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

In [12]:
train_predictions

array(['per', 'geo', 'eve', ..., 'geo', 'gpe', 'geo'], dtype=object)

In [13]:
train_y

566     per
183     geo
2130    eve
5776    org
1706    gpe
       ... 
6728    per
1794    per
3409    geo
5862    gpe
4026    geo
Name: NE_class, Length: 5538, dtype: object

In [14]:
test_predictions

array(['geo', 'gpe', 'tim', ..., 'per', 'org', 'gpe'], dtype=object)

In [15]:
test_y

3       per
4       geo
12      geo
21      geo
29      gpe
       ... 
6902    org
6911    org
6915    per
6916    geo
6920    per
Name: NE_class, Length: 1384, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [16]:
test_y #1384 'gold' NE classes 

3       per
4       geo
12      geo
21      geo
29      gpe
       ... 
6902    org
6911    org
6915    per
6916    geo
6920    per
Name: NE_class, Length: 1384, dtype: object

In [17]:
test_predictions #1384 preducted NE classes

# confusion matrix: N*N ie 8*8

array(['geo', 'gpe', 'tim', ..., 'per', 'org', 'gpe'], dtype=object)

In [18]:
a2.confusion_matrix(test_y, test_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,3,0,0,1,0,0,0,0
eve,0,1,1,1,0,0,0,0
geo,3,2,195,56,0,61,44,87
gpe,2,1,41,86,0,36,40,19
nat,0,0,0,0,2,0,0,0
org,3,0,54,35,1,78,31,23
per,0,0,37,44,0,46,98,26
tim,1,5,59,21,1,32,24,83


In [19]:
a2.confusion_matrix(train_y, train_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,39,0,0,0,0,0,1,0
eve,0,28,0,1,0,0,0,0
geo,0,2,1506,72,2,80,47,65
gpe,0,0,50,807,0,38,35,17
nat,0,0,0,0,14,0,0,0
org,1,0,46,39,0,803,19,17
per,1,5,38,49,0,42,752,20
tim,0,1,43,18,0,21,16,803


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [22]:
a2.bonusb('GMB_dataset.txt', reduced_dims=3000)

Traing:Testing = 80.0%:19.999999999999996% 
Reduced dims: 3000-D 


 Confusion matrix of TEST DATA:
     art  eve  geo  gpe  nat  org  per  tim
art    1    0    0    0    0    0    0    1
eve    0    0    0    0    0    0    0    0
geo    2    3  243   55    0   66   51   65
gpe    2    1   47  100    0   44   37   24
nat    0    0    0    0    0    0    0    0
org    3    2   49   26    0   75   35   20
per    0    1   39   34    0   31   91   16
tim    4    4   39   24    2   28   22   97

 Confusion matrix of TRAINING DATA:
     art  eve   geo  gpe  nat  org  per  tim
art   38    0     0    0    0    0    0    0
eve    0   26     0    0    0    0    0    0
geo    1    4  1478   78    3   96   38   73
gpe    0    0    49  808    0   45   18   27
nat    0    0     0    0   14    0    0    0
org    1    2    42   40    1  786   32   17
per    0    1    37   42    0   43  763   21
tim    1    1    47   23    0   23   20  799


In [24]:
a2.bonusb('GMB_dataset.txt', reduced_dims=4000)

Traing:Testing = 80.0%:19.999999999999996% 
Reduced dims: 4000-D 


 Confusion matrix of TEST DATA:
     art  eve  geo  gpe  nat  org  per  tim
art    2    0    0    1    0    0    0    0
eve    0    0    3    0    0    0    0    0
geo    8    3  220   57    3   70   41   67
gpe    1    1   50   98    0   37   25   18
nat    0    0    0    0    0    0    0    0
org    2    2   50   38    1   75   33   24
per    0    1   31   37    0   34  108   20
tim    3    3   39   29    1   20   18  110

 Confusion matrix of TRAINING DATA:
     art  eve   geo  gpe  nat  org  per  tim
art   36    0     0    0    0    0    0    0
eve    0   30     1    0    0    0    0    0
geo    0    2  1516   66    3   81   42   65
gpe    0    0    44  803    0   51   19   20
nat    0    0     0    0   12    0    0    0
org    0    0    41   31    0  801   26   16
per    0    2    42   51    0   45  782   17
tim    1    1    33   19    0   23   13  803
